In [1]:
#All Packages Used
import pandas as pd
import h2o
import numpy as np
from h2o.estimators.random_forest import H2ORandomForestEstimator

In [2]:
#Import Data
test = pd.read_csv('test.csv',parse_dates=True,index_col='Date')
data = pd.read_csv('train.csv',parse_dates=True,index_col='Date')
store = pd.read_csv('store.csv')
states = pd.read_csv('store_states.csv')

/Users/LiamRoberts/anaconda3/envs/DS1/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
#Convert Date Columns
def add_dates(data):
    data['Year'] = data.index.year
    data['Month'] = data.index.month
    data['Day'] = data.index.day
    data['WeekOfYear'] = data.index.weekofyear
    return data
test = add_dates(test)
data = add_dates(data)

#Fill NaN values for store
#store.drop(columns =['CompetitionOpenSinceMonth',
                     #'CompetitionOpenSinceYear',
                     #'Promo2SinceWeek',
                     #'Promo2SinceYear','PromoInterval','CompetitionDistance'],inplace = True)

In [5]:
#Merge training dataframe
pd_df = data.merge(store,on='Store')
pd_df = pd_df.merge(states,on='Store')
pd_df['Store'] = pd_df['Store'].astype('category')

#Merge Testin Dataframe
test_df = test.merge(store,on='Store')
test_df = test_df.merge(states,on='Store')
test_df['Store'] = test_df['Store'].astype('category')

#Drop times when store didnt make enough sales (these will also cause NaN values for log transforms)
pd_df = pd_df[(pd_df.Sales>0)]

In [6]:
pd_df.head()

,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Year,Month,...,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,State
0,1,5,5263,555,1,1,0,1,2015,7,...,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,HE
1,1,4,5020,546,1,1,0,1,2015,7,...,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,HE
2,1,3,4782,523,1,1,0,1,2015,7,...,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,HE
3,1,2,5011,560,1,1,0,1,2015,7,...,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,HE
4,1,1,6102,612,1,1,0,1,2015,7,...,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,HE


In [7]:
#Make sure test data is in order for prediction
test_df.set_index('Id',inplace = True)
test_df.sort_index(inplace = True)

In [8]:
test_df.head()

,Store,DayOfWeek,Open,Promo,StateHoliday,SchoolHoliday,Year,Month,Day,WeekOfYear,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,State
Id,,,,,,,,,,,,,,,,,,,,
1,1,4,1.0,1,0,0,2015,9,17,38,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,HE
2,3,4,1.0,1,0,0,2015,9,17,38,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct",NW
3,7,4,1.0,1,0,0,2015,9,17,38,a,c,24000.0,4.0,2013.0,0,NaN,NaN,NaN,SH
4,8,4,1.0,1,0,0,2015,9,17,38,a,a,7520.0,10.0,2014.0,0,NaN,NaN,NaN,SH
5,9,4,1.0,1,0,0,2015,9,17,38,a,c,2030.0,8.0,2000.0,0,NaN,NaN,NaN,NW


In [9]:
#Initialize h2o cluster and clear any previously saved info from the cluster
h2o.init(nthreads=-1,max_mem_size='6G')
h2o.remove_all()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_192"; Java(TM) SE Runtime Environment (build 1.8.0_192-b12); Java HotSpot(TM) 64-Bit Server VM (build 25.192-b12, mixed mode)
  Starting server from /Users/LiamRoberts/anaconda3/envs/DS1/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/sj/qvzvqx5n1qv59yg5v3pz2y400000gn/T/tmp9on4l7wh
  JVM stdout: /var/folders/sj/qvzvqx5n1qv59yg5v3pz2y400000gn/T/tmp9on4l7wh/h2o_LiamRoberts_started_from_python.out
  JVM stderr: /var/folders/sj/qvzvqx5n1qv59yg5v3pz2y400000gn/T/tmp9on4l7wh/h2o_LiamRoberts_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,America/Toronto
H2O data parsing timezone:,UTC
H2O cluster version:,3.20.0.7
H2O cluster version age:,2 months and 13 days
H2O cluster name:,H2O_from_python_LiamRoberts_bbvoj3
H2O cluster total nodes:,1
H2O cluster free memory:,5.333 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


In [10]:
#Create h2o frame for training models
train = h2o.H2OFrame(python_obj=pd_df)
testh2o = h2o.H2OFrame(python_obj=test_df)

/Users/LiamRoberts/anaconda3/envs/DS1/lib/python3.6/site-packages/h2o/utils/shared_utils.py:177: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  data = _handle_python_lists(python_obj.as_matrix().tolist(), -1)[1]


Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [13]:
print(testh2o.shape)
testh2o.head(3)

(41088, 20)


Store,DayOfWeek,Open,Promo,StateHoliday,SchoolHoliday,Year,Month,Day,WeekOfYear,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,State
1,4,1,1,0,0,2015,9,17,38,c,a,1270,9,2008,0,nan,nan,nan,HE
3,4,1,1,0,0,2015,9,17,38,a,a,14130,12,2006,1,14,2011,"Jan,Apr,Jul,Oct",NW
7,4,1,1,0,0,2015,9,17,38,a,c,24000,4,2013,0,nan,nan,nan,SH


h2o's Random Distributed Models will automatically encode categorical variables, which saves the trouble of having to process them before hand. The Random Distributed Forest also has built in ways to deal with missing values allowing it to provide predictions very quickly "out of the box".

In [14]:
#Log transform sales data and assign X and y columns names
train['log_sales'] = train['Sales'].log()
X_labels = [i for i in train.col_names if (i not in ['Sales','Customers','log_sales'])]
y_labels = 'log_sales'

In [15]:
#Create Model
model = H2ORandomForestEstimator(    
    ntrees=100,
    max_depth = 30,
    nbins_cats = 1115,
    seed = 1)

#h2o supports .fit() to fit into the sklearn pipeline however recommends using .train() 
#and passing the full dataframe into the model
model.train(x=X_labels,y=y_labels,training_frame=train)

drf Model Build progress: |███████████████████████████████████████████████| 100%


In [16]:
#Get predictions and transfrom them back to untransformed state
predictions = model.predict(testh2o)
pred = predictions.expm1()

drf prediction progress: |████████████████████████████████████████████████| 100%


In [18]:
pred.shape

(41088, 1)

In [19]:
#Convert h2o frame back into a pandas data frame for data manipulation and CSV conversion
submission = pred.as_data_frame()

In [20]:
#Convert Data Frame to CSV for submission
submission['Id'] = test['Id'].values
submission['Sales'] = submission['expm1(predict)']
submission.set_index('Id',inplace = True)
submission.drop(columns = 'expm1(predict)',inplace = True)
submission.to_csv('h2osub.csv')

In [21]:
#This line will prevent the cluster from getting its memory too filled up if the notebook is run multiple times
h2o.cluster().shutdown()

H2O session _sid_968c closed.
